In [32]:
pip install -q datasets sentence-transformers faiss-cpu accelerate langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-c5oi6i44
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-c5oi6i44
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 55.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from transformers import LlamaForCausalLM, LlamaTokenizerFast       # LLM for report classificuing
from sentence_transformers import SentenceTransformer       # for embedding model
from sklearn.model_selection import train_test_split
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, pipeline#, AutoModelForSeq2SeqGeneration
# from transformers import AutoModelForSeq2SeqGeneration

In [ ]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

In [34]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show transformers

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/nvolk/anaconda3/lib/python3.12/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoModelForSeq2SeqLM


In [11]:
filename = "data/labeled_data_combined_reports.csv"

In [12]:
# load data in df format
df_reports = pd.read_csv(filename)
df_reports.head(2)

,Unnamed: 0,patient_id,imaging_date,reports,image_ct___1,image_ct___2,image_ct___3,combined_reports
0,0,SHSC-134CJ-PV3YY-9L6O6-PLRA9OVTHG-4JZ2M-UR0UO-...,2010-09-21,Bone Scan(Whole Body)Nuc Med TECHNETIUM MDP BO...,0.0,1.0,0.0,Bone Scan(Whole Body)Nuc Med TECHNETIUM MDP BO...
1,1,SHSC-134CJ-PV3YY-9L6O6-PLRA9OVTHG-4JZ2M-UR0UO-...,2011-01-13,CT Chest History:\rFollow-up scan for OZM-011 ...,1.0,1.0,0.0,Bone Scan(Whole Body)Nuc Med TECHNETIUM MDP BO...


In [13]:
df_reports["report_and_frac_label"] = (
    "Report:\n" + 
    df_reports["combined_reports"] + 
    "\n\nFracture classification:\n" + 
    df_reports["image_ct___1"]
    # df_reports["image_ct___1"].apply(lambda x: "Positive" if float(x) > 0 else "Negative")
)

df_reports["report_and_mets_label"] = (
    "Report:\n" + 
    df_reports["combined_reports"] + 
    "\n\nMetastases classification:\n" + 
    df_reports["image_ct___1"]
    # df_reports["image_ct___2"].apply(lambda x: "Positive" if float(x) > 0 else "Negative")
)

# drop reports that have NaN in reports column
df_reports = df_reports.dropna(subset=["report_and_frac_label", "report_and_mets_label"])

df_reports.head(2)


,Unnamed: 0,patient_id,imaging_date,reports,image_ct___1,image_ct___2,image_ct___3,combined_reports,report_and_frac_label,report_and_mets_label
0,0,SHSC-134CJ-PV3YY-9L6O6-PLRA9OVTHG-4JZ2M-UR0UO-...,2010-09-21,Bone Scan(Whole Body)Nuc Med TECHNETIUM MDP BO...,0.0,1.0,0.0,Bone Scan(Whole Body)Nuc Med TECHNETIUM MDP BO...,Report:\nBone Scan(Whole Body)Nuc Med TECHNETI...,Report:\nBone Scan(Whole Body)Nuc Med TECHNETI...
1,1,SHSC-134CJ-PV3YY-9L6O6-PLRA9OVTHG-4JZ2M-UR0UO-...,2011-01-13,CT Chest History:\rFollow-up scan for OZM-011 ...,1.0,1.0,0.0,Bone Scan(Whole Body)Nuc Med TECHNETIUM MDP BO...,Report:\nBone Scan(Whole Body)Nuc Med TECHNETI...,Report:\nBone Scan(Whole Body)Nuc Med TECHNETI...


In [ ]:
class RAGPipeline:
    def __init__(self, 
                 embedding_model_name="sentence-transformers/all-mpnet-base-v2",
                 llm_model_name="google/flan-t5-large",  # Example seq2seq model
                 device="cuda" if torch.cuda.is_available() else "cpu"):
        
        print(f"Initializing RAG system on {device}")
        
        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        self.tokenizer = AutoTokenizer.from_pretrained(
            llm_model_name,
            trust_remote_code=True
        )
        
        # load LLM
        print("Loading language model...")
        model = AutoModelForSeq2SeqLM.from_pretrained(
            llm_model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto",
            trust_remote_code=True,
            load_in_8bit=True if device == "cuda" else False
        )
        
        # text gen pipeline
        pipe = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=self.tokenizer,
            max_length=512,
            temperature=0.7,
            top_p=0.95,
            # device=device,
            do_sample=True
        )
        
        self.llm = HuggingFacePipeline(pipeline=pipe)
        self.vectorstore = None
        self.qa_chain = None

    def _create_prompt(self, context, question):
        return f"""Answer the following question based on the given context.

Context: {context}

Question: {question}

Answer:"""

    def load_data(self, df, text_column):

        loader = DataFrameLoader(df, page_content_column=text_column)
        documents = loader.load()
        
        texts = self.text_splitter.split_documents(documents)
        self.vectorstore = FAISS.from_documents(texts, self.embeddings)
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(
                search_kwargs={"k": 3}
            )
        )
        
        return f"Loaded {len(texts)} text chunks into the vector store"

    def query(self, question):
        """query RAG system"""
        if self.qa_chain is None:
            raise ValueError("Please load data first")
        
        # retrieve relevant docs
        docs = self.vectorstore.similarity_search(question, k=3)
        context = "\n\n".join([doc.page_content for doc in docs])
        
        prompt = self._create_prompt(context, question)
        
        response = self.llm(prompt)
        
        return response[0]['generated_text'] if isinstance(response, list) else response

    def batch_query(self, questions):
        """be able to handle multiple reports at once"""
        return [self.query(q) for q in questions]

    def similarity_search(self, query, k=3):
        if self.vectorstore is None:
            raise ValueError("Please load data first")
        return self.vectorstore.similarity_search(query, k=k)

    def save_vectorstore(self, path):
        """save FAISS vector database locally"""
        if self.vectorstore:
            self.vectorstore.save_local(path)
            print(f"Vector database saved to {path}")
        else:
            raise ValueError("No vector store to save")

    def load_vectorstore(self, path):
        """Load a saved FAISS vector store"""
        print(f"Loading vector store from {path}")
        self.vectorstore = FAISS.load_local(path, self.embeddings)
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(
                search_kwargs={"k": 3}
            )
        )

In [ ]:
rag = RAGPipeline(
        # embedding_model_name="sentence-transformers/all-mpnet-base-v2",
        # llm_model_name="meta-llama/Llama-2-7b-chat-hf"  # Or any other causal LM
    )

Initializing RAG system on cuda


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading tokenizer...
Loading language model...


/tmp/ipykernel_1075279/1967846821.py:57: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  self.llm = HuggingFacePipeline(pipeline=pipe)


In [19]:
print("Loading data...")
result = rag.load_data(df_reports, text_column='report_and_frac_label')
print(result)

Loading data...
Loaded 22227 text chunks into the vector store


In [22]:
input = "..."
print(f"Input: {input}")
output = rag.query(input)


Input: ...


In [24]:
print(f"LLM Output: {output}")

LLM Output: Report


In [ ]:
# batch reports & LLM labels
inputs = [
    "..."
]
outputs = rag.batch_query(inputs)

for i, o in zip(inputs, outputs):
    print(f"\Input: {i}")
    print(f"Output: {o}")